In [1]:
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [2]:
import numpy as np
from glob import glob
import pandas as pd
from tqdm import tqdm
from interfacegan_utils import calculate_boundary

In [3]:
latent_input_base_path = f"{DATA_PATH}/Models/InterfaceGAN/Inputs/e4e_00005_concatenated/"
all_paths = glob(f"{latent_input_base_path}/*/*")
all_paths = [path for path in all_paths if not "summary_stats" in path]

In [4]:
for path in tqdm(all_paths):
    out_dir = path.replace('Inputs', 'Outputs')
    if os.path.exists(out_dir):
        continue

    else:
        accuracies = []
        os.makedirs(out_dir, exist_ok=True)
        latent_codes_path = f"{path}/latents_concatenated.npy"
        scores_path = f"{path}/target.npy"
        out_name = f'boundary_dim.npy'
        acc = calculate_boundary(out_dir, out_name, latent_codes_path, scores_path)
        accuracies.append(acc)

        # save result
        attribute = path.split('/')[-2]
        values = path.split('/')[-1].split('_')

        # Crate dataframe with columns 'attribute', 'value1, 'value2', 'accuracy1', 'accuracy2', ...
        result = pd.DataFrame(columns=['attribute'] + ['neg_class', 'pos_class'] + ['acc'])
        result.loc[0] = [attribute] + values + accuracies
        result.to_csv(f"{out_dir}/summary_stats.csv", index=False)

100%|██████████| 982/982 [00:00<00:00, 233426.27it/s]


In [5]:
summary_paths = glob(f"{latent_input_base_path.replace('Inputs', 'Outputs')}/*/*/summary_stats.csv")

training_results = pd.concat([pd.read_csv(path) for path in summary_paths])
training_results

,attribute,neg_class,pos_class,acc
0,sleeve_length,Sleeveless,Elbow length,0.998008
0,sleeve_length,Extra short,Elbow length,0.997722
0,sleeve_length,34 length,Mixed,0.979920
0,sleeve_length,Mixed,Extra long,0.968468
0,sleeve_length,Extra short,Strapless,0.985240
...,...,...,...,...
0,pattern,Checked,Pinstriped,0.888889
0,pattern,Animal print,Striped,0.924603
0,pattern,Floral,Checked,0.973357
0,pattern,Plain,Print,0.952519
